AGE AND GENDER DETECTION IN A MEETING ROOM

In [1]:
import cv2
import numpy as np
import os

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

Using the UTKFace dataset for age and gender detection

In [3]:
# Path to the UTKFace dataset
UTK = "UTKFace"


In [9]:
# Prepare data
data = []
gender_labels = []
age_labels = []

for img_name in os.listdir(UTK):
    img_path = os.path.join(UTK, img_name)
    img = cv2.imread(img_path)
    img = cv2.resize(img, (64, 64))
    
    age_label = int(img_name.split("_")[0])  # Age label
    gender_label = int(img_name.split("_")[1])  # Gender label
    
    data.append(img)
    gender_labels.append(gender_label)
    age_labels.append(age_label)

data = np.array(data, dtype="float") / 255.0
gender_labels = np.array(gender_labels)
age_labels = np.array(age_labels)


In [10]:
# Split the data
(trainX, testX, trainGenderY, testGenderY) = train_test_split(data, gender_labels, test_size=0.2, random_state=42)
(trainX, testX, trainAgeY, testAgeY) = train_test_split(data, age_labels, test_size=0.2, random_state=42)


In [11]:
# Build the gender classification model
gender_model = Sequential()
gender_model.add(Conv2D(32, (3, 3), padding="same", activation="relu", input_shape=(64, 64, 3)))
gender_model.add(MaxPooling2D(pool_size=(2, 2)))
gender_model.add(Conv2D(64, (3, 3), padding="same", activation="relu"))
gender_model.add(MaxPooling2D(pool_size=(2, 2)))
gender_model.add(Flatten())
gender_model.add(Dense(128, activation="relu"))
gender_model.add(Dropout(0.5))
gender_model.add(Dense(1, activation="sigmoid"))


In [12]:
# Compile the gender model
gender_model.compile(loss="binary_crossentropy", optimizer=Adam(0.001), metrics=["accuracy"])

In [13]:
# Train the gender model
gender_model.fit(trainX, trainGenderY, validation_data=(testX, testGenderY), batch_size=32, epochs=10, verbose=1)


Epoch 1/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 32s 51ms/step - accuracy: 0.6984 - loss: 0.5734 - val_accuracy: 0.8596 - val_loss: 0.3176
Epoch 2/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 29s 49ms/step - accuracy: 0.8561 - loss: 0.3253 - val_accuracy: 0.8809 - val_loss: 0.2684
Epoch 3/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 28s 47ms/step - accuracy: 0.8811 - loss: 0.2789 - val_accuracy: 0.8874 - val_loss: 0.2589
Epoch 4/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 28s 47ms/step - accuracy: 0.8898 - loss: 0.2649 - val_accuracy: 0.8846 - val_loss: 0.2702
Epoch 5/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 28s 47ms/step - accuracy: 0.8965 - loss: 0.2403 - val_accuracy: 0.8943 - val_loss: 0.2443
Epoch 6/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 28s 48ms/step - accuracy: 0.9117 - loss: 0.2173 - val_accuracy: 0.8975 - val_loss: 0.2407
Epoch 7/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 28s 48ms/step - accuracy: 0.9026 - loss: 0.2208 - val_accuracy: 0.8984 - val_loss: 0.2511
Epoch 8/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 29s 49ms/step - accuracy: 0.9166 - loss: 0.1956 - 

In [14]:
# Save the gender model
gender_model.save("gender_classification_model.h5")

In [15]:
# Build the age detection model
age_model = Sequential()
age_model.add(Conv2D(32, (3, 3), padding="same", activation="relu", input_shape=(64, 64, 3)))
age_model.add(MaxPooling2D(pool_size=(2, 2)))
age_model.add(Conv2D(64, (3, 3), padding="same", activation="relu"))
age_model.add(MaxPooling2D(pool_size=(2, 2)))
age_model.add(Flatten())
age_model.add(Dense(128, activation="relu"))
age_model.add(Dropout(0.5))
age_model.add(Dense(1, activation="linear"))  # Linear activation for age regression


In [16]:
# Compile the age model
age_model.compile(loss="mean_squared_error", optimizer=Adam(0.001), metrics=["mae"])


In [17]:
# Train the age model
age_model.fit(trainX, trainAgeY, validation_data=(testX, testAgeY), batch_size=32, epochs=10, verbose=1)

Epoch 1/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 1965s 3s/step - loss: 420.6267 - mae: 15.5906 - val_loss: 183.2208 - val_mae: 10.1478
Epoch 2/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 31s 52ms/step - loss: 210.7736 - mae: 10.9990 - val_loss: 154.7344 - val_mae: 9.2144
Epoch 3/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 28s 47ms/step - loss: 176.8996 - mae: 10.0405 - val_loss: 155.8442 - val_mae: 9.1225
Epoch 4/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 28s 47ms/step - loss: 165.2999 - mae: 9.6416 - val_loss: 120.0314 - val_mae: 8.0316
Epoch 5/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 28s 47ms/step - loss: 147.7255 - mae: 9.0786 - val_loss: 118.4013 - val_mae: 7.8741
Epoch 6/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 28s 48ms/step - loss: 134.5982 - mae: 8.6564 - val_loss: 117.4112 - val_mae: 7.8542
Epoch 7/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 29s 48ms/step - loss: 128.9658 - mae: 8.3994 - val_loss: 140.2838 - val_mae: 8.5267
Epoch 8/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 30s 50ms/step - loss: 126.7289 - mae: 8.3848 - val_loss: 122.7733 - val_mae: 7.8921
Epoc

In [18]:
# Save the age model
age_model.save("age_detection_model.h5")

In [19]:
# Paths to images and masks
image_dir = "C:/Users/SARATHLAL/Downloads/null class projects/3.meeting_room_det/DeepFashion_In-shop_Clothes_Retrieval_Adjusted/images"
mask_dir = "C:/Users/SARATHLAL/Downloads/null class projects/3.meeting_room_det/DeepFashion_In-shop_Clothes_Retrieval_Adjusted/masks"
output_dir = "output_dataset"

In [20]:
# Create output directories if they don't exist
os.makedirs(os.path.join(output_dir, "white"), exist_ok=True)
os.makedirs(os.path.join(output_dir, "black"), exist_ok=True)
os.makedirs(os.path.join(output_dir, "other"), exist_ok=True)


In [21]:
def segment_shirt(image, mask):
    # Apply the mask to the image
    segmented = cv2.bitwise_and(image, image, mask=mask)
    return segmented


In [22]:
def detect_shirt_color(segmented_shirt):
    # Convert segmented shirt to HSV
    hsv_img = cv2.cvtColor(segmented_shirt, cv2.COLOR_BGR2HSV)
    
    # Define color ranges for white and black
    mask_white = cv2.inRange(hsv_img, (0, 0, 200), (180, 20, 255))
    mask_black = cv2.inRange(hsv_img, (0, 0, 0), (180, 255, 30))
    
    white_pixels = cv2.countNonZero(mask_white)
    black_pixels = cv2.countNonZero(mask_black)
    
    if white_pixels > black_pixels:
        return "white"
    elif black_pixels > white_pixels:
        return "black"
    else:
        return "other"

# Process each image and mask
for img_name in os.listdir(image_dir):
    img_path = os.path.join(image_dir, img_name)
    mask_path = os.path.join(mask_dir, img_name)
    
    image = cv2.imread(img_path)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    
    if image is not None and mask is not None:
        # Segment the shirt region
        segmented_shirt = segment_shirt(image, mask)
        
        # Detect the shirt color
        shirt_color = detect_shirt_color(segmented_shirt)
        
        # Save the segmented shirt image in the appropriate directory
        output_path = os.path.join(output_dir, shirt_color, img_name)
        cv2.imwrite(output_path, segmented_shirt)

In [ ]:
train the Shirt Color Classification Model

In [23]:
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

In [29]:
# Paths to dataset
dataset_path = "C:/Users/SARATHLAL/Downloads/null class projects/3.meeting_room_det/DeepFashion_In-shop_Clothes_Retrieval_Adjusted/images"


In [30]:
# Parameters
img_height, img_width = 64, 64
batch_size = 32
epochs = 10

In [32]:
# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)


Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


In [18]:
# Get the number of classes
num_classes = len(train_generator.class_indices)

In [19]:
# Build the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')  # Update to use num_classes
])

In [20]:
# Compile the model
model.compile(optimizer=Adam(0.001), loss='categorical_crossentropy', metrics=['accuracy'])


In [21]:
# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_steps=validation_generator.samples // batch_size,
    validation_data=validation_generator,
    epochs=epochs
)

Epoch 1/10


C:\Users\SARATHLAL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


461/461 ━━━━━━━━━━━━━━━━━━━━ 120s 253ms/step - accuracy: 0.9811 - loss: 0.0315 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/10
461/461 ━━━━━━━━━━━━━━━━━━━━ 0s 177us/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/10


C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


461/461 ━━━━━━━━━━━━━━━━━━━━ 94s 202ms/step - accuracy: 1.0000 - loss: 4.3101e-07 - val_accuracy: 1.0000 - val_loss: 1.9436e-10
Epoch 4/10
461/461 ━━━━━━━━━━━━━━━━━━━━ 0s 143us/step - accuracy: 1.0000 - loss: 3.7253e-09 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/10
461/461 ━━━━━━━━━━━━━━━━━━━━ 93s 201ms/step - accuracy: 1.0000 - loss: 1.5774e-08 - val_accuracy: 1.0000 - val_loss: 6.4788e-11
Epoch 6/10
461/461 ━━━━━━━━━━━━━━━━━━━━ 0s 108us/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 7/10
461/461 ━━━━━━━━━━━━━━━━━━━━ 94s 202ms/step - accuracy: 1.0000 - loss: 1.3485e-08 - val_accuracy: 1.0000 - val_loss: 2.9154e-10
Epoch 8/10
461/461 ━━━━━━━━━━━━━━━━━━━━ 0s 102us/step - accuracy: 1.0000 - loss: 1.4901e-08 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 9/10
461/461 ━━━━━━━━━━━━━━━━━━━━ 95s 205ms/step - accuracy: 1.0000 - loss: 1.8764e-08 - val_accuracy: 1.0000 - val_loss: 3.2394e-11
Epoch 10/10
461/461 ━━━━━━━━━━━━━━━━━━━━ 

In [22]:
# Save the model
model.save("shirt_color_classification_model.h5")

In [23]:

# Paths to the model files
weights_path = "DeepFashion_In-shop_Clothes_Retrieval_Adjusted/yolov3.weights"  # Path to yolov3 weights file
config_path = "DeepFashion_In-shop_Clothes_Retrieval_Adjusted/yolov3.cfg"       # Path to yolov3 config file
labels_path = "DeepFashion_In-shop_Clothes_Retrieval_Adjusted/coco.names"   

In [24]:
# Load the COCO class labels
with open(labels_path, 'r') as f:
    labels = f.read().strip().split("\n")
print(labels)

['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


In [25]:
# Load YOLOv3 model
net = cv2.dnn.readNetFromDarknet(config_path, weights_path)
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]



In [26]:
from tensorflow.keras.models import load_model
# Load the pre-trained gender classification model
gender_model = load_model("gender_classification_model.h5")


In [27]:
# Load the pre-trained shirt color classification model
shirt_color_model = load_model("shirt_color_classification_model.h5")

In [32]:
# Image path
image_path = "C:/Users/SARATHLAL/Downloads/null class projects/DeepFashion_In-shop_Clothes_Retrieval_Adjusted/meeting room.jpeg"

In [35]:
# Check if the file exists
if not os.path.isfile(image_path):
    print(f"Error: File '{image_path}' not found.")
else:
    # Load the image
    image = cv2.imread(image_path)
    
    if image is None:
        print(f"Error: Failed to load image '{image_path}'.")
    else:
        # Get image dimensions
        height, width = image.shape[:2]

        # Prepare the image for YOLO
        blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), swapRB=True, crop=False)
        net.setInput(blob)
        detections = net.forward(output_layers)

        # Initialize lists to hold the detection results
        boxes = []
        confidences = []
        class_ids = []

        # Process each detection
        for output in detections:
            for detection in output:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if labels[class_id] == "person" and confidence > 0.5:
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)
                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)

        # Apply non-maxima suppression to filter the detections
        indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

        # Only proceed if there are at least 2 people detected
        if len(indices) >= 2:
            for i in indices.flatten():
                x, y, w, h = boxes[i]
                person_img = image[y:y+h, x:x+w]

                # Resize the person image for gender classification
                person_img_resized = cv2.resize(person_img, (64, 64))
                person_img_resized = np.expand_dims(person_img_resized, axis=0) / 255.0

                # Gender classification
                gender_pred = gender_model.predict(person_img_resized)
                gender = 'Male' if gender_pred[0][0] > 0.5 else 'Female'

                # Resize the person image for shirt color classification
                person_img_resized = cv2.resize(person_img, (64, 64))
                person_img_resized = np.expand_dims(person_img_resized, axis=0) / 255.0

                # Shirt color classification
                shirt_color_pred = shirt_color_model.predict(person_img_resized)
                shirt_color = np.argmax(shirt_color_pred, axis=1)[0]

                if shirt_color == 0:  # Assuming 0 is white
                    age = 23
                    shirt_color_str = "White"
                elif shirt_color == 1:  # Assuming 1 is black
                    age = "Child"
                    shirt_color_str = "Black"
                else:
                    age = "Unknown"
                    shirt_color_str = "Other"

                color = (0, 255, 0)  # Green color for bounding box
                label = f"{gender}, Age: {age}, Shirt: {shirt_color_str}"
                cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
                cv2.putText(image, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # Display the output image
        cv2.imshow("Detected People", image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
